In [ ]:
%matplotlib inline

# Example plot for LFPy: Single-synapse contribution to the LFP

Copyright (C) 2017 Computational Neuroscience Group, NMBU.

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

In [ ]:
import LFPy
import numpy as np
import os
import sys
from urllib.request import urlopen
import ssl
import zipfile
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
from os.path import join

## Fetch Mainen&Sejnowski 1996 model files:

In [ ]:
if not os.path.isfile(join('cells', 'cells', 'j4a.hoc')):
    #get the model files:
    u = urlopen('http://senselab.med.yale.edu/ModelDB/eavBinDown.asp?o=2488&a=23&mime=application/zip',
                context=ssl._create_unverified_context())
    localFile = open('patdemo.zip', 'wb')
    localFile.write(u.read())
    localFile.close()
    #unzip:
    myzip = zipfile.ZipFile('patdemo.zip', 'r')
    myzip.extractall('.')
    myzip.close()

## Main script, set parameters and create cell, synapse and electrode objects:

In [ ]:
# Define cell parameters
cell_parameters = {
    'morphology' : join('cells', 'cells', 'j4a.hoc'), # from Mainen & Sejnowski, J Comput Neurosci, 1996
    'cm' : 1.0,         # membrane capacitance
    'Ra' : 150.,        # axial resistance
    'v_init' : -65.,    # initial crossmembrane potential
    'passive' : True,   # turn on NEURONs passive mechanism for all sections
    'passive_parameters' : {'g_pas' : 1./30000, 'e_pas' : -65},
    'nsegs_method' : 'lambda_f', # spatial discretization method
    'lambda_f' : 100.,           # frequency where length constants are computed
    'dt' : 2.**-3,      # simulation time step size
    'tstart' : 0.,      # start time of simulation, recorders start at t=0
    'tstop' : 100.,     # stop simulation at 100 ms.
}

# Create cell
cell = LFPy.Cell(**cell_parameters)
# Align cell
cell.set_rotation(x=4.98919, y=-4.33261, z=0.)

In [ ]:
# Define synapse parameters
synapse_parameters = {
    'idx' : cell.get_closest_idx(x=0., y=0., z=900.),
    'e' : 0.,                   # reversal potential
    'syntype' : 'ExpSyn',       # synapse type
    'tau' : 10.,                # syn. time constant
    'weight' : .001,            # syn. weight
    'record_current' : True,
}

# Create synapse and set time of synaptic input
synapse = LFPy.Synapse(cell, **synapse_parameters)
synapse.set_spike_times(np.array([20.]))

In [ ]:
# Run simulation, record transmembrane currents
cell.simulate(rec_imem=True)

In [ ]:
# Create a grid of measurement locations, in (mum)
X, Z = np.mgrid[-500:501:50, -400:1201:50]
Y = np.zeros(X.shape)

# Define electrode parameters
grid_electrode_parameters = {
    'sigma' : 0.3,      # extracellular conductivity
    'x' : X.flatten(),  # electrode requires 1d vector of positions
    'y' : Y.flatten(),
    'z' : Z.flatten()
}

# Create electrode objects
grid_electrode = LFPy.RecExtElectrode(cell, **grid_electrode_parameters)

# Calculate LFPs as product between linear transform and currents. 
# Create reference to data on class object
grid_electrode.LFP = grid_electrode.get_transformation_matrix() @ cell.imem

## Plot simulation output:

In [ ]:
fig = plt.figure(dpi=160, figsize=(12, 6))

ax = fig.add_axes([.1,.1,.25,.8], frameon=False)
ax.axis('off')
ax1 = fig.add_axes([.35,.1,.6,.8], aspect='auto', frameon=False)
ax1.axis('off')

from matplotlib.textpath import TextPath
from matplotlib.font_manager import FontProperties
from matplotlib.patches import PathPatch

fp = FontProperties(family="Courier New", weight='roman')
path = TextPath((0, 0), 'LFPy', prop=fp, size=50, )

ax1.add_patch(PathPatch(path, facecolor='C1', clip_on=False, linewidth=3))
ax1.axis([0, 120, -15, 35])

LFP = grid_electrode.LFP[:, cell.tvec==30].reshape(X.shape)
linthresh = 1E-5
vmax = 2E-4

C0 = plt.cm.colors.hex2color('k')
C1 = plt.cm.colors.hex2color('C4')
C2 = plt.cm.colors.hex2color('C1')
C3 = plt.cm.colors.hex2color('C3')
C4 = plt.cm.colors.hex2color('k')
cmap = plt.cm.colors.LinearSegmentedColormap.from_list('C0C1C2', colors=[C0, C1, C2, C3, C4], N=256, gamma=1)

im = ax.contourf(X, Z, LFP,
                 norm=plt.cm.colors.SymLogNorm(linthresh=linthresh, linscale=1,
                                               vmin=-vmax, vmax=vmax),
                 levels=101,
                 cmap=cmap,
                 zorder=-2)

#plot morphology
zips = []
for x, z in cell.get_idx_polygons():
    zips.append(list(zip(x, z)))
polycol = PolyCollection(zips,
                         edgecolors='k',
                         facecolors='k',
                         lw=0.2)
ax.add_collection(polycol)

ax.plot(synapse.x, synapse.z, 'o', ms=10,
        markeredgecolor='k',
        markerfacecolor='C1')

fig.savefig('LFPy-logo.png', bbox_inches='tight')
fig.savefig('LFPy-logo.svg', bbox_inches='tight')